# Project TFX Pipeline - Mental Health Prediction

## Import Library

In [126]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.dsl.components.common.resolver import Resolver
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.types import Channel
import pandas as pd
import os
import json

## Setup Environment

In [67]:
PIPELINE_NAME = "nabilsaragih-pipeline"
SCHEMA_PIPELINE_NAME = "nabilsaragih-tfdv-schema"

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [68]:
df = pd.read_csv('data/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51093 entries, 0 to 51092
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  51093 non-null  object
 1   status     51093 non-null  object
dtypes: object(2)
memory usage: 798.5+ KB


In [69]:
df.head()

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless heart. All out of tune",Anxiety
2,"All wrong, back off dear, forward doubt. Stay in a restless and restless place",Anxiety
3,I've shifted my focus to something else but I'm still worried,Anxiety
4,"I'm restless and restless, it's been a month now, boy. What do you mean?",Anxiety


In [70]:
DATA_ROOT = 'data'

In [71]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

### ExampleGen
---
Mengonfigurasi pembagian data (data split) dalam komponen ExampleGen pada pipeline TFX. `output` didefinisikan sebagai instance dari `example_gen_pb2.Output` yang menerima konfigurasi pembagian melalui parameter `split_config`. Dalam konfigurasi ini, data dibagi menjadi dua bagian: "train" dengan 8 hash buckets (80% data) dan "eval" dengan 2 hash buckets (20% data), menggunakan metode hash untuk memastikan pembagian yang deterministik dan konsisten.

In [72]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

Menjalankan komponen CsvExampleGen untuk memasukkan data berformat CSV ke dalam pipeline TFX. Komponen ini diinisialisasi dengan parameter input_base yang mengarah ke direktori `DATA_ROOT` dan `output_config` yang menentukan cara pembagian data (seperti train dan eval). Selanjutnya, `interactive_context.run(example_gen)` digunakan untuk mengeksekusi komponen tersebut secara interaktif, menghasilkan TFRecord yang sudah dibagi sesuai konfigurasi untuk tahap selanjutnya dalam pipeline machine learning.

In [73]:
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

### StatisticsGen
---
Menjalankan komponen `StatisticsGen` dalam pipeline TFX untuk menghasilkan statistik deskriptif dari data yang telah diproses oleh ExampleGen. Komponen `StatisticsGen` diinisialisasi dengan input berupa `examples`, yaitu output data dari `example_gen`. Kemudian, `interactive_context.run(statistic_gen)` mengeksekusi komponen ini secara interaktif, menghasilkan ringkasan statistik seperti distribusi nilai, frekuensi, nilai hilang, dan lainnya. Statistik ini berguna untuk eksplorasi data awal dan akan digunakan oleh komponen lain seperti SchemaGen untuk mendeteksi skema data secara otomatis.

In [74]:
statistic_gen = StatisticsGen(examples=example_gen.outputs["examples"])
interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [75]:
interactive_context.show(statistic_gen.outputs['statistics'])

### SchemaGen
---
Menjalankan komponen `SchemaGen` dalam pipeline TFX untuk menghasilkan skema data secara otomatis berdasarkan statistik yang telah dihitung sebelumnya. Komponen `SchemaGen` diinisialisasi dengan input `statistics` yang berasal dari output `StatisticsGen`, yaitu ringkasan statistik dari data. Selanjutnya, `interactive_context.run(schema_gen)` mengeksekusi proses ini secara interaktif. Hasilnya adalah skema data (schema), yang mencakup informasi seperti jenis data (tipe), apakah fitur bersifat wajib (required), nilai maksimum/minimum, dan struktur lainnya. Skema ini penting untuk proses validasi dan mendeteksi anomali data di tahap berikutnya.

In [76]:
schema_gen = SchemaGen(
    statistics=statistic_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [77]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'statement',BYTES,required,,-
'status',STRING,required,,'status'


,Values
Domain,
'status',"'Anxiety', 'Bipolar', 'Depression', 'Normal', 'Personality disorder', 'Stress', 'Suicidal'"


### ExampleValidator
---
Menjalankan komponen `ExampleValidator` dalam pipeline TFX untuk melakukan validasi data terhadap skema yang telah ditentukan. Komponen `ExampleValidator` diinisialisasi dengan dua input: `statistics` dari output `StatisticsGen` dan `schema` dari output `SchemaGen`. Kemudian, `interactive_context.run(example_validator)` mengeksekusi komponen ini secara interaktif. Tujuannya adalah untuk mendeteksi anomali dalam data, seperti nilai yang hilang, tipe data yang tidak sesuai, atau nilai yang berada di luar rentang yang diharapkan. Hasil validasi ini sangat penting untuk memastikan kualitas dan konsistensi data sebelum digunakan dalam proses pelatihan model machine learning.


In [78]:
example_validator = ExampleValidator(
    statistics=statistic_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [79]:
interactive_context.show(example_validator.outputs["anomalies"])

## Data Preprocessing

### Transform
---
Komponen `Transform` digunakan untuk menerapkan transformasi data menggunakan fungsi `preprocessing_fn` yang didefinisikan dalam file eksternal `transform.py`. Komponen ini menerima data contoh (`examples`) dari `ExampleGen` dan skema (`schema`) dari `SchemaGen`. File `transform.py` berisi fungsi preprocessing yang mendefinisikan dua transformasi: pertama, mengubah nilai label `status` menjadi representasi numerik melalui `StaticHashTable` (misalnya 'Anxiety' menjadi 0, 'Depression' menjadi 2, dst.); kedua, mengubah fitur teks `statement` menjadi huruf kecil menggunakan `tf.strings.lower()`. Transformasi ini dibungkus ke dalam dictionary output dengan nama fitur yang diubah diberi akhiran `_xf`. Eksekusi dilakukan dengan `interactive_context.run(transform)`, menghasilkan data yang sudah diproses dan siap digunakan untuk pelatihan model secara konsisten pada training dan serving time.


In [ ]:
TRANSFORM_MODULE_FILE = "modules/transform.py"
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Model Development

### Tuner
---
Komponen `Tuner` digunakan untuk mencari konfigurasi hyperparameter terbaik bagi model klasifikasi teks dalam pipeline TFX. Komponen ini menerima data yang telah ditransformasi (`transformed_examples`), grafik transformasi (`transform_graph`), dan skema (`schema`) serta file Python eksternal `tuner.py` yang mendefinisikan proses tuning. Di dalam `tuner.py`, fungsi `tuner_fn` bertanggung jawab membangun pipeline tuning. Data pelatihan dan evaluasi dibaca dengan `input_fn`, yang memanfaatkan hasil transformasi dari `TFTransformOutput`. Model dibangun menggunakan `model_builder`, yang memanfaatkan lapisan `TextVectorization`, `Embedding`, dan beberapa lapisan dense/dropout. Keras Tuner `RandomSearch` digunakan untuk menguji berbagai kombinasi jumlah unit neuron, dropout, dan learning rate. Model dikompilasi dengan `sparse_categorical_crossentropy` karena labelnya berupa integer. Komponen ini akan secara otomatis menjalankan beberapa eksperimen (hingga 20 percobaan) dan memilih model dengan `val_accuracy` terbaik, yang hasilnya akan digunakan pada tahap pelatihan model.


In [ ]:
TUNER_MODULE_FILE = "modules/tuner.py"
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),
)
interactive_context.run(tuner)

Trial 20 Complete [00h 04m 12s]
val_accuracy: 0.741252601146698

Best val_accuracy So Far: 0.7434087991714478
Total elapsed time: 01h 26m 58s
Results summary
Results in pipelines\nabilsaragih-pipeline\.temp\10\mental_health_classifier
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 03 summary
Hyperparameters:
unit_1: 64
dropout_1: 0.30000000000000004
unit_2: 128
dropout_2: 0.5
learning_rate: 0.0005
Score: 0.7434087991714478

Trial 00 summary
Hyperparameters:
unit_1: 256
dropout_1: 0.5
unit_2: 128
dropout_2: 0.5
learning_rate: 0.0005
Score: 0.7419386506080627

Trial 11 summary
Hyperparameters:
unit_1: 128
dropout_1: 0.4
unit_2: 64
dropout_2: 0.5
learning_rate: 0.001
Score: 0.7419386506080627

Trial 19 summary
Hyperparameters:
unit_1: 128
dropout_1: 0.30000000000000004
unit_2: 64
dropout_2: 0.5
learning_rate: 0.001
Score: 0.741252601146698

Trial 09 summary
Hyperparameters:
unit_1: 256
dropout_1: 0.30000000000000004
unit_2: 128
dropout_2: 0.4
learning_rate: 

ExecutionResult(
    component_id: Tuner
    execution_id: 10
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### Trainer
---
Komponen `Trainer` digunakan untuk melatih model klasifikasi berdasarkan data yang telah diproses oleh komponen `Transform` dan hasil tuning dari komponen `Tuner`. Di dalam file `trainer.py`, fungsi `run_fn` mengatur seluruh proses pelatihan model. Data pelatihan dan validasi dibaca dari TFRecord yang telah ditransformasi, dan dilakukan adaptasi teks menggunakan `TextVectorization`. Arsitektur model dibangun melalui `model_builder`, yang mendukung pilihan penggunaan LSTM atau pooling biasa, tergantung hasil tuning. Model dikompilasi dengan loss `sparse_categorical_crossentropy` dan optimizer Adam dengan learning rate yang dapat disesuaikan. Proses pelatihan dilengkapi dengan callback seperti `TensorBoard`, `EarlyStopping`, `ModelCheckpoint`, dan `ReduceLROnPlateau`. Selain itu, `class_weight` dihitung otomatis untuk menangani ketidakseimbangan kelas. Setelah pelatihan, model disimpan beserta *serving signature* melalui fungsi `_get_serve_tf_examples_fn`, yang memungkinkan data mentah ditransformasi secara otomatis saat deployment. Model yang dihasilkan siap digunakan untuk prediksi dalam lingkungan produksi.


In [ ]:
TRAINER_MODULE_FILE = "modules/trainer.py"
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),
)
interactive_context.run(trainer)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 statement_xf (InputLayer)   [(None, 1)]               0         
                                                                 
 tf.reshape_3 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization_3 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 64)           640000    
                                                                 
 global_average_pooling1d_4   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_11 (Dense)            (None, 64)                4160

INFO:tensorflow:Assets written to: pipelines\nabilsaragih-pipeline\Trainer\model\14\Format-Serving\assets


1000/1000 [==============================] - 23s 22ms/step - loss: 2.7427 - accuracy: 0.5195 - val_loss: 1.1680 - val_accuracy: 0.5713 - lr: 5.0000e-04
Epoch 2/15
 998/1000 [============================>.] - ETA: 0s - loss: 2.6504 - accuracy: 0.6119
Epoch 2: val_accuracy did not improve from 0.57133
1000/1000 [==============================] - 19s 19ms/step - loss: 2.6466 - accuracy: 0.6120 - val_loss: 0.9446 - val_accuracy: 0.5703 - lr: 5.0000e-04
Epoch 3/15
1000/1000 [==============================] - ETA: 0s - loss: 1.5538 - accuracy: 0.6658
Epoch 3: val_accuracy improved from 0.57133 to 0.63878, saving model to pipelines\nabilsaragih-pipeline\Trainer\model\14\Format-Serving
INFO:tensorflow:Assets written to: pipelines\nabilsaragih-pipeline\Trainer\model\14\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\nabilsaragih-pipeline\Trainer\model\14\Format-Serving\assets


1000/1000 [==============================] - 20s 20ms/step - loss: 1.5538 - accuracy: 0.6658 - val_loss: 0.8818 - val_accuracy: 0.6388 - lr: 5.0000e-04
Epoch 4/15
 998/1000 [============================>.] - ETA: 0s - loss: 1.4235 - accuracy: 0.7102
Epoch 4: val_accuracy improved from 0.63878 to 0.65930, saving model to pipelines\nabilsaragih-pipeline\Trainer\model\14\Format-Serving
INFO:tensorflow:Assets written to: pipelines\nabilsaragih-pipeline\Trainer\model\14\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\nabilsaragih-pipeline\Trainer\model\14\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\nabilsaragih-pipeline\Trainer\model\14\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 14
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Resolver
---
Komponen `Resolver` digunakan untuk mengambil model terakhir yang telah *diberkati* (blessed) sebagai referensi dalam evaluasi model baru. Dengan menggunakan strategi `LatestBlessedModelStrategy`, komponen ini mencari model terbaru yang sudah lulus validasi kualitas (biasanya melalui komponen Evaluator). Parameter `model` dan `model_blessing` didefinisikan sebagai `Channel` untuk `Model` dan `ModelBlessing`, yaitu output dari pelatihan dan evaluasi sebelumnya. Pemberian ID `Latest_blessed_model_resolver` memudahkan identifikasi komponen ini dalam pipeline. Eksekusi dengan `interactive_context.run(model_resolver)` memungkinkan pipeline secara otomatis mendapatkan versi model terbaik terakhir sebagai pembanding untuk menentukan apakah model baru layak di-*deploy*.


In [95]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 15
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Evaluator
---
Komponen `Evaluator` digunakan untuk mengevaluasi model yang telah dilatih dengan cara membandingkan kinerjanya terhadap model sebelumnya (baseline) yang telah diberkati. Fungsi `get_eval_config()` menyusun konfigurasi evaluasi dengan mendefinisikan label (`status_xf`) dan prediksi (`predictions`), serta metrik-metrik evaluasi seperti `SparseCategoricalAccuracy`, `Precision`, `Recall`, `AUC`, dan metrik kesalahan seperti `FalsePositives` dan `FalseNegatives`. Salah satu metrik (`SparseCategoricalAccuracy`) dilengkapi dengan ambang batas minimum (≥ 0.5) dan perubahan akurasi minimum dari model sebelumnya. Komponen ini juga menggunakan `SlicingSpec` untuk menghitung metrik secara keseluruhan. Saat `Evaluator` dijalankan melalui `interactive_context.run(evaluator)`, TFX akan menghasilkan evaluasi menyeluruh dan memutuskan apakah model baru cukup baik untuk di-*blessing*, yang menjadi syarat sebelum proses deployment dilakukan.


In [ ]:
def get_eval_config():
    return tfma.EvalConfig(
        model_specs=[
            tfma.ModelSpec(
                label_key='status_xf',
                prediction_key='predictions'
            )
        ],
        slicing_specs=[tfma.SlicingSpec()],
        metrics_specs=[
            tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='SparseCategoricalAccuracy'),
                tfma.MetricConfig(class_name='Precision', config=json.dumps({'name': 'macro'})),
                tfma.MetricConfig(class_name='Recall', config=json.dumps({'name': 'macro'})),
                tfma.MetricConfig(class_name='AUC', config=json.dumps({'name': 'macro'})),
                tfma.MetricConfig(class_name='FalsePositives'),
                tfma.MetricConfig(class_name='FalseNegatives'),
                tfma.MetricConfig(
                    class_name='SparseCategoricalAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.5}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': 0.0001}
                        )
                    )
                )
            ])
        ]
    )


In [121]:
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=get_eval_config()
)
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 26
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

### TFMA Visualization
---
Setelah proses evaluasi model selesai dijalankan, kode ini digunakan untuk memuat dan menampilkan hasilnya secara visual. Baris `eval_result = evaluator.outputs['evaluation'].get()[0].uri` mengambil lokasi output hasil evaluasi dari komponen `Evaluator`. Selanjutnya, `tfma_result = tfma.load_eval_result(eval_result)` memuat hasil evaluasi tersebut ke dalam objek Python yang bisa dianalisis. Visualisasi metrik performa model, seperti akurasi atau AUC, dilakukan dengan `tfma.view.render_slicing_metrics(tfma_result)`, yang menyajikan metrik dalam bentuk tabel interaktif, termasuk kemungkinan *slicing* per fitur jika dikonfigurasi. Terakhir, `tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)` menampilkan `Fairness Indicator`, yaitu widget tambahan dari TFMA untuk mengevaluasi keadilan model terhadap kelompok data tertentu (misalnya ras, gender, dll), sangat penting untuk memastikan tidak ada bias dalam hasil prediksi model.


In [125]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

## Model Deployment

### Pusher
---
Komponen `Pusher` digunakan untuk meng-*deploy* model terlatih ke lokasi penyimpanan tertentu apabila model tersebut telah melewati evaluasi dan mendapatkan status *blessing* dari komponen `Evaluator`. Pada kode tersebut, model hasil pelatihan (`trainer.outputs['model']`) akan didorong ke folder `serving_model/mental-health-model` jika evaluasi menyatakan model layak digunakan (`evaluator.outputs['blessing']`). Direktori tujuan disediakan melalui `PushDestination`, yang dalam hal ini menggunakan sistem berkas lokal (filesystem). Eksekusi `interactive_context.run(pusher)` akan memindahkan model ke folder yang ditentukan, sehingga dapat digunakan untuk serving, baik melalui REST API, aplikasi prediksi, maupun integrasi produksi lainnya. Ini merupakan langkah terakhir dalam pipeline TFX untuk menyiapkan model yang siap pakai.


In [127]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory="serving_model/mental-health-model"
        )
    )
)
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 27
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))